## UCSF

The files are not aligned. Align and extract.
Note the ligand may be in the other chain or in both.


In [ ]:
#folder = '/Users/matteo/Dropbox/File requests/Macrodomain'
folder = '20200928_ucsf_pdbs'
import os, re, pymol2
from fragmenstein import Victor
from rdkit import Chem

In [ ]:
prefix = 'UCSF2'

In [ ]:
os.mkdir(f'{prefix}-aligned')
os.mkdir(f'{prefix}-hits')

In [ ]:
import pandas as pd

master = pd.read_excel('20200928_ucsf_fraglist.xlsx')
smilesdex = {row.Dataset_ID: row.SMILES for i, row in master.iterrows()}

# the following are manually overridden.
smilesdex['mac-x0570'] = 'CC(C)(C)C=1C=C(*)C(O)=C(O)C1'
smilesdex['mac-x0453'] = 'NCCC1=CNC=N1'
smilesdex['mac-x0139'] = 'CN(CCC(=O)O)C=1N=CN=C2NC=CC12'

In [ ]:
for file in os.listdir(folder):
    if '.pdb' not in file:
        continue
    # name
    name = file.replace('.pdb', '')
    if os.path.exists(f'{prefix}-hits/{name}.mol'):
        #continue
        pass
    ## Align
    #print(f'Aligning {name}')
    with pymol2.SingletonPyMOL() as pymol:
        pymol.cmd.load(os.path.join(folder, file), name)
        pymol.cmd.load('template.pdb', 'template')
        pymol.cmd.remove('element H')
        pymol.cmd.remove('solvent')
        if not pymol.cmd.select(name='keeper', selection=f'{name} and resn LIG'):
            raise ValueError(f'There is no residue LIG in {name}')
        elif pymol.cmd.select(name='keeper', selection=f'{name} and chain A and resn LIG'):
            # ligand is chain A and is its polymer.
            pass
        elif pymol.cmd.get_model(f'({name} and resn LIG) around 5').atom[0].chain == 'A':
            # ligand is not chain A but its polymer is
            pymol.cmd.alter(f'{name} and resn LIG', "chain='A'")
            pymol.cmd.sort()
        else:
            # neither is chain A.
            pymol.cmd.remove(f'chain A and {name}')
            pymol.cmd.alter(name, "chain='A'")
            pymol.cmd.sort()
        pymol.cmd.remove(f'{name} and not chain A')
        pymol.cmd.align(name, 'template')
        pymol.cmd.remove("not alt ''+A")
        pymol.cmd.alter('all', "alt=''")
        pymol.cmd.save(f'{prefix}-aligned/{name}.pdb', selection=f'{name} and chain A')
    ## Extract
    #print(f'Extracting {name}')
    mol = Victor.extract_mol(name=name, 
                             smiles=smilesdex[name],
                             filepath=f'{prefix}-aligned/{name}.pdb', 
                             removeHs=True)
    mol.SetProp('_Name', name)
    Chem.MolToMolFile(mol, f'{prefix}-hits/{name}.mol')